In [1]:
# To Save back
# http://www.countingcalculi.com/explanations/google_sheets_and_jupyter_notebooks/
# from df2gspread import df2gspread as d2g
# wks_name = 'Jupyter Manipulated Data'
# d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

In [49]:
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
# use Seaborn styles
%matplotlib inline
sns.set() 

In [3]:
# Authorise with Google Docs
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/data/docs/JupyterSheets.json', scope)
gc = gspread.authorize(credentials)

In [4]:
# Load Spreadsheet
spreadsheet_key = '1KQwNxNVYJ21K1LrDcAddEj1_yxSWlxVpc6xS9z542yY'
book = gc.open_by_key(spreadsheet_key)
uk = pd.DataFrame(book.worksheet("UKTxns").get_all_records())
us = pd.DataFrame(book.worksheet("USTxns").get_all_records())

In [5]:
def sort_tables(df):
    cols = ['Account', 'Date', 'Payee', 'Type', 'Category', 'Memo', 'Outflow', 'Inflow', 'Net']
    df['Date'] = pd.to_datetime(df['Date'])
    df['Inflow'] = pd.to_numeric(df['Inflow'])
    df['Outflow'] = pd.to_numeric(df['Outflow'])
    df['Inflow'].fillna(value=0, inplace=True)
    df['Outflow'].fillna(value=0, inplace=True)
    df['Net'] = df['Inflow'] - df ['Outflow']
    #df['Net'] = pd.to_numeric(df['Net'])
    df = df[cols]
    return df

In [6]:
def add_date_info(df):
    """
    Adds columns for year, month, quarter, week number
    """
    df['Year'], df['Month'] = df['Date'].dt.year, df['Date'].dt.month
    df['Quarter'], df['Week'] = df['Date'].dt.quarter, df['Date'].dt.week
    #df['Year'] = df['Year'].astype('int')
    #df['Month'] = df['Month'].astype('int')
    #df['Week'] = df['Week'].astype('int')
    #df['Quarter'] = df['Quarter'].astype('int')

In [7]:
sort_tables(uk)
add_date_info(uk)
#cols = ['Account', 'Date', 'Payee', 'Type', 'Category', 'Memo', 'Outflow', 'Inflow', 'Net']
#UK = UK[cols]

In [8]:
cols = ['Account', 'Date', 'Payee', 'Type', 'Category', 'Memo', 'Outflow', 'Inflow', 'Net']
uk = uk[cols]
add_date_info(uk)

In [9]:
uk.head()

,Account,Date,Payee,Type,Category,Memo,Outflow,Inflow,Net,Year,Month,Quarter,Week
0,VJ Cash ISA,2014-06-29,Starting Balance,Transfer,,,0.00,11628.24,11628.24,2014,6,2,26
1,AJ Cash ISA,2014-06-30,Starting Balance,Transfer,,,0.00,11629.02,11629.02,2014,6,2,27
2,AJ Cash ISA,2014-11-20,HMRC,Expense,Zisc,Interest removed,25.02,0.00,-25.02,2014,11,4,47
3,Nationwide,2015-01-01,Starting Balance,Transfer,,,0.00,11079.15,11079.15,2015,1,1,1
4,Halifax CC,2015-01-02,Starting Balance,Transfer,,,839.56,0.00,-839.56,2015,1,1,1


In [65]:
def f(row):
    expenses = ['Expense', 'NBC', 'Expat']
    if row['Type'] in expenses:
        val = 'Expense'
    else:
        val = row['Type']
    return val

In [69]:
filtered = uk.query('Type != "Transfer" & Year != "2014"')
uktype = pd.pivot_table(filtered, index='Type', columns='Year', values='Net',
                        aggfunc=sum, fill_value=0, margins=True)
uktype

Year,2015,2016,2017,2018,All
Type,,,,,
Expat,-33324.73,-22077.68,-36142.11,-7365.94,-98910.46
Expense,-14662.88,-24349.24,-21223.20,-10377.86,-70613.18
Income,88300.19,89347.90,114151.92,69128.52,360928.53
Invest,-12000.00,-11000.00,-86000.02,-48600.00,-157600.02
NBC,-3544.68,-4475.86,-3049.73,-2353.45,-13423.72
All,24767.90,27445.12,-32263.14,431.27,20381.15


In [54]:
#uktype.loc['Expenses'] = uktype.loc['Expense'] + uktype.loc['Expat'] + uktype.loc['NBC']
#uktype

Year,2015,2016,2017,2018,All
Type,,,,,
Expat,-33324.73,-22077.68,-36142.11,-7365.94,-98910.46
Expense,-14662.88,-24349.24,-21223.20,-10377.86,-70613.18
Income,88300.19,89347.90,114151.92,69128.52,360928.53
Invest,-12000.00,-11000.00,-86000.02,-48600.00,-157600.02
NBC,-3544.68,-4475.86,-3049.73,-2353.45,-13423.72
All,24767.90,27445.12,-32263.14,431.27,20381.15
Expenses,-51532.29,-50902.78,-60415.04,-20097.25,-182947.36
